<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/38_3_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка необходимых пакетов
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libcudnn8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires keras>=3.5.0, which is not installed.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires keras>=3.5.0, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00


In [2]:
# Импорт необходимых библиотек
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text
import re
import pathlib
import random
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Загрузка датасета
examples, metadata = tfds.load('web_questions', with_info=True)
train_examples, test_examples = examples['train'], examples['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3778 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.85XPAQ_1.0.0/web_questions-train.tfrecord*...:   …

Generating test examples...:   0%|          | 0/2032 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.85XPAQ_1.0.0/web_questions-test.tfrecord*...:   0…

Dataset web_questions downloaded and prepared to /root/tensorflow_datasets/web_questions/1.0.0. Subsequent calls will reuse this data.


In [4]:
print('Размер обучающей выборки: ', len(train_examples))
print('Размер валидационной выборки: ', len(test_examples))

Размер обучающей выборки:  3778
Размер валидационной выборки:  2032


In [5]:
for q_examples, a_examples in train_examples.batch(3).take(1):
    print('Примеры на португальском языке:')
    print()
    for q in q_examples.numpy():
        print(q.decode('utf-8'))
    print()

    print('Примеры на английском языке:')
    print()
    for en in a_examples.numpy():
        print(a.decode('utf-8'))

ValueError: too many values to unpack (expected 2)

In [6]:
train_question = train_examples.map(lambda x: x['question'])
train_answer = train_examples.map(lambda x: x['answers'][0])

print(train_question)
print(train_answer)

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [7]:
# Токенизация списков вопросов и ответов
VOCAB_SIZE = 8000
bert_tokenizer_params=dict(lower_case=True)

reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    vocab_size = VOCAB_SIZE,
    reserved_tokens=reserved_tokens,
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},)

In [8]:
question_vocab = bert_vocab.bert_vocab_from_dataset(train_question.batch(1000).prefetch(2), **bert_vocab_args)
print(question_vocab[:10])
print(question_vocab[100:110])
print(question_vocab[1000:1010])
print(question_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', "'", '(', ')', '.', '0', '1']
['use', 'college', '##ie', 'and', 'city', 'movies', '##k', '##h', '##m', 'state']
[]
['##3', '##4', '##5', '##6', '##7', '##8', '##9', '##?', '##j', '##q']


In [9]:
def write_vocab_file(filepath, vocab):
    with open(filepath, 'w') as f:
        for token in vocab:
            print(token, file=f)

write_vocab_file('question_vocab.txt', question_vocab)

In [10]:
answer_vocab = bert_vocab.bert_vocab_from_dataset(train_answer.batch(1000).prefetch(2),  **bert_vocab_args)
print(answer_vocab[:10])
print(answer_vocab[100:110])
print(answer_vocab[1000:1010])
print(answer_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '&', "'", '(']
['york', '##ist', 'city', '##c', '##ch', '##ie', 'in', '##ion', 'canada', 'north']
[]
['##.', '##/', '##:', '##?', '##j', '##p', '##q', '##£', '##–', '##−']


In [11]:
write_vocab_file('answer_vocab.txt', answer_vocab)

In [12]:
!ls *.txt

answer_vocab.txt  question_vocab.txt


In [13]:
question_tokenizer = text.BertTokenizer('question_vocab.txt', **bert_tokenizer_params)
answer_tokenizer = text.BertTokenizer('answer_vocab.txt', **bert_tokenizer_params)

In [14]:
# Токенизируем примеры и получаем тензор формы (batch, word, word-piece)
token_batch = answer_tokenizer.tokenize(train_answer)
# Объединяем оси word и word-piece и получаем тензор формы (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

print('Токенизируем отобранные строки:')
for ex in token_batch.to_list():
  print(ex)

ValueError: Attempt to convert a value (<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>) with an unsupported type (<class 'tensorflow.python.data.ops.map_op._MapDataset'>) to a Tensor.